In [1]:
%%time

#import in all libraries and methods
%run -i 'myimports.py'
%run -i 'datagathering.py'
%run -i 'nlp_processing.py' #imports packages and methods
main = "https://www.happyscribe.com"                
url="/public/lex-fridman-podcast-artificial-intelligence-ai"
podcast_host = "Lex Fridman"
html_location = "Archive/happyscribe_Lex_Fridman_html.txt"


/public/lex-fridman-podcast-artificial-intelligence-ai/145-matthew-johnson-psychedelics
/public/lex-fridman-podcast-artificial-intelligence-ai/144-michael-littman-reinforcement-learning-and-the-future-of-ai
/public/lex-fridman-podcast-artificial-intelligence-ai/143-john-clarke-the-art-of-fighting-and-the-pursuit-of-excellence
/public/lex-fridman-podcast-artificial-intelligence-ai/142-manolis-kellis-meaning-of-life-the-universe-and-everything
/public/lex-fridman-podcast-artificial-intelligence-ai/141-erik-brynjolfsson-economics-of-ai-social-networks-and-technology
/public/lex-fridman-podcast-artificial-intelligence-ai/140-lisa-feldman-barrett-love-evolution-and-the-human-brain
/public/lex-fridman-podcast-artificial-intelligence-ai/139-andrew-huberman-neuroscience-of-optimal-performance
/public/lex-fridman-podcast-artificial-intelligence-ai/138-yaron-brook-ayn-rand-and-the-philosophy-of-objectivism
/public/lex-fridman-podcast-artificial-intelligence-ai/137-alex-filippenko-supernovae-dark

In [10]:
update_transcripts(html_location, main)

In [2]:
doc = nlp(open("3Lex/#101|Joscha_Bach|Artificial_Consciousness_and_the_Nature_of_Reality.txt").read())
doc.user_data["summary"]

/ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


['And eventually the idea is let us not make an assumption about this thing if you do not know what the right solution is and those areas that you have no idea whether the right solution will be people designing this ad hoc or machines doing this, whether you want to enforce compliance by social norms like wikipedia or orvis software solutions or the a.i. that goes through the posts of people or is a legal principle',
 'And they look through the world, through the lens of their phones and think it is the real world, but it is the twitter world that is distorted by the popularity incentives of twitter.',
 'So i think putting computation at the center of the world view is actually the right way to think about it.',
 'And there is been some successes, as you brought up with go with alpha go, alpha zero, with ideas of self play, which i think are incredibly interesting ideas of systems playing each other and in an automated way, to improve by playing other systems of in a particular constr

In [3]:
doc.user_data

{'summary': ['And eventually the idea is let us not make an assumption about this thing if you do not know what the right solution is and those areas that you have no idea whether the right solution will be people designing this ad hoc or machines doing this, whether you want to enforce compliance by social norms like wikipedia or orvis software solutions or the a.i. that goes through the posts of people or is a legal principle',
  'And they look through the world, through the lens of their phones and think it is the real world, but it is the twitter world that is distorted by the popularity incentives of twitter.',
  'So i think putting computation at the center of the world view is actually the right way to think about it.',
  'And there is been some successes, as you brought up with go with alpha go, alpha zero, with ideas of self play, which i think are incredibly interesting ideas of systems playing each other and in an automated way, to improve by playing other systems of in a pa

In [2]:
%%time
#convert file of podcasts to list of docs and docbin and write to disk

# %run -i 'nlp_implementation.py'
podcast_name = "Lex_Fridman_Podcast"
# os.listdir(podcast_name)
onlyfiles = folder_to_filelist(podcast_name) #get list of all files in foldername

doc_bin = DocBin(store_user_data=True) #docbin container for serialization
docs = [] #list of docs 
print("starting")
for doc, name in nlp.pipe(onlyfiles, as_tuples=True): #piping all collection of docs to make doclist and docbin
    #each doc contains hostname, guest, title, entities mentioned, and summary
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    
    docs.append(doc)
    doc_bin.add(doc) #add doc to list and bin
    
with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
    f.write(doc_bin.to_bytes())
print("done")

starting


/ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


done
CPU times: user 1min 32s, sys: 34.5 s, total: 2min 7s
Wall time: 2min 44s


In [6]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))
# with open("cucumber/first_doc.bin", "rb") as w: #loading in
#     new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
# docs1 += list(new_docs1.get_docs(nlp.vocab))
for doc in docs1:
    print(doc.user_data["title"], doc.user_data["guest"])
# print(docs1[5].user_data["books"])

Love, Evolution, and the Human Brain Lisa Feldman Barrett
Quantum Computing Scott Aaronson
Economics of AI, Social Networks, and Technology Erik Brynjolfsson
Microsoft CTO Kevin Scott
Psychedelics Matthew Johnson
Reinforcement Learning and the Future of AI Michael Littman
Deep Learning, Education, and Real-World AI Andrew Ng
The Hard Problem of Consciousness David Chalmers
Cruise Automation Kyle Vogt
Thinking Fast and Slow, Deep Learning, and AI Daniel Kahneman
Effective Altruism William MacAskill
IBM Watson, Jeopardy & Deep Conversations with AI David Ferrucci
Space Exploration, Space Suits, and Life on Mars Dava Newman
Algorithms, TeX, Life, and The Art of Computer Programming Donald Knuth
Neuralink, AI, Autopilot, and the Pale Blue Dot Elon Musk
Tesla Autopilot Elon Musk
The Art of Fighting and the Pursuit of Excellence John Clarke
Universal Artificial Intelligence, AIXI, and AGI Marcus Hutter
Physics View of the Mind and Neurobiology John Hopfield
Stack Overflow and Coding Horror J

In [ ]:
#for each category of ents, generate csv files for ents corresponding to people

att_to_csv(docs1, "people") 
# att_to_csv(docs1, "places")
# att_to_csv(docs1, "books")
# print(sorted(set([item for sublist in docs1 for item in sublist.user_data["people"]])))

In [ ]:
#generate megagraph for book ents
edges = pd.read_csv("booksEdges.csv", sep=',').drop("id", axis=1).values.tolist()
nodes = pd.read_csv('booksNodes.csv',sep=',').set_index("id").to_dict("index")

source = [nodes.get(e[0])["name"] for e in edges]
target = [nodes.get(e[1])["name"] for e in edges]
kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':np.ones(len(source))})
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

#following are other example graphviews

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['target']=="Bible"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['source']=="Matthew Johnson"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['source']=="Matthew Johnson" )| (kg_df['source']=="Ryan Hall")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['target']=="Crime and Punishment" )| (kg_df['target']=="Atlas Shrugged")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()